In [1]:
from sshtunnel import SSHTunnelForwarder 
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os
import pandas as pd
import plotly.express as px

In [2]:
load_dotenv()

True

Need to install MariaDB Connector/C from CS Package Repository https://mariadb.com/docs/server/connect/programming-languages/c/install/

Need to install MariaDB community server on the machine

In [3]:
def run_query(query_str):
    with SSHTunnelForwarder(
        (os.getenv("BASTION_SERVER_IP"), 22), #Remote server IP and SSH port
        ssh_username = os.getenv("SSH_USER_NAME"),
        ssh_pkey=os.getenv("SSH_PRIVATE_KEY_PATH"),
        remote_bind_address=(os.getenv("RDS_ENDPOINT"), 3306)) as server: 
            
        server.start() #start ssh sever
        # print('Server connected via SSH')
        
        #connect to MariaDB
        local_port = str(server.local_bind_port)
        database_name = 'financial'
        engine = create_engine(
            'mariadb+mariadbconnector://' 
            + os.getenv('DB_USERNAME') 
            + ':' 
            + os.getenv('DB_PASSWORD') 
            + '@127.0.0.1:'
            + local_port 
            + '/' 
            + database_name
        )

        Session = sessionmaker(bind=engine)
        session = Session()
        
        # print('Database session created')
        
        #test data retrieval
        test = session.execute(text(query_str))
        df = pd.DataFrame.from_records(test)
            
        session.close()

        return df

### List Tables

In [4]:
df = run_query('SHOW TABLES')
df

,0
0,account
1,card
2,client
3,disp
4,district
5,loan
6,order
7,trans


### Account 

In [17]:
df = run_query('SHOW COLUMNS FROM account')
df

,0,1,2,3,4,5
0,account_id,int(11),NO,PRI,0,
1,district_id,int(11),NO,MUL,0,
2,frequency,varchar(18),NO,,None,
3,date,date,NO,,None,


In [18]:
df = run_query('SELECT * FROM account LIMIT 5')
df

,0,1,2,3
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [19]:
df = run_query('SELECT DISTINCT frequency FROM account')
df

,0
0,POPLATEK MESICNE
1,POPLATEK TYDNE
2,POPLATEK PO OBRATU


The data is in Czech.
- POPLATEK MESICNE: Monthly Issuance
- POPLATEK TYDNE: Weekly Issuance
- POPLATEK PO OBRATU: Issuance After Transaction

In [20]:
df = run_query('SELECT MIN(date) earliest_date, MAX(date) latest_date FROM account')
df

,0,1
0,1993-01-01,1997-12-29


In [4]:
df = run_query('SELECT district_id, COUNT(*) count FROM account GROUP BY district_id')
df.columns = ['district_id', 'count']
df

,district_id,count
0,1,554
1,2,42
2,3,50
3,4,48
4,5,65
...,...,...
72,73,56
73,74,135
74,75,51
75,76,55


In [6]:
fig = px.histogram(df, x='district_id', y='count')
fig.show()

Most of the account is from district 0 to 9

In [7]:
df = run_query('SELECT frequency, COUNT(*) count FROM account GROUP BY frequency')
df.columns = ['frequency', 'count']
fig = px.histogram(df, x='frequency', y='count')
fig.show()

Most of the account's frequency is POPLATEK MESICNE(MONTHLY FEE)

In [4]:
df = run_query("SELECT month, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(date, '%Y-%m-01') AS DATETIME) month FROM account) tb GROUP BY month")
df.columns = ['month', 'count']

In [5]:
fig = px.bar(df, x='month', y='count')
fig.show()

Most of the account is created in 1993 and 1996

### Card

In [10]:
df = run_query('SHOW COLUMNS FROM card')
df

,0,1,2,3,4,5
0,card_id,int(11),NO,PRI,0,
1,disp_id,int(11),NO,MUL,None,
2,type,varchar(7),NO,,None,
3,issued,date,NO,,None,


In [12]:
df = run_query('SELECT type, COUNT(*) count FROM card GROUP BY type')
df.columns = ['type', 'count']
fig = px.histogram(df, x='type', y='count')
fig.show()

Most of the cards are Classic type.

In [13]:
df = run_query("SELECT issued_year, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(issued, '%Y-01-01') AS DATETIME) issued_year FROM card) tb GROUP BY issued_year")
df.columns = ['issued_year', 'count']
fig = px.bar(df, x='issued_year', y='count')
fig.show()

Most of the cards are issued in 1998

### Client

In [4]:
df = run_query('SHOW COLUMNS FROM client')
df

,0,1,2,3,4,5
0,client_id,int(11),NO,PRI,None,
1,gender,varchar(1),NO,,None,
2,birth_date,date,NO,,None,
3,district_id,int(11),NO,MUL,None,


In [6]:
df = run_query('SELECT gender, COUNT(*) count FROM client GROUP BY gender')
df.columns = ['gender', 'count']
fig = px.histogram(df, x='gender', y='count')
fig.show()

The clients are equally distributed between males and females

In [5]:
df = run_query("SELECT birth_year, COUNT(*) FROM (SELECT CAST(DATE_FORMAT(birth_date, '%Y-01-01') AS DATETIME) birth_year FROM client) tb GROUP BY birth_year")
df.columns = ['birth_year', 'count']
fig = px.bar(df, x='birth_year', y='count')
fig.show()

Most of the clients' birth year are between 1939 to 1980

In [7]:
df = run_query("SELECT gender, birth_year, COUNT(*) FROM (SELECT gender, CAST(DATE_FORMAT(birth_date, '%Y-01-01') AS DATETIME) birth_year FROM client) tb GROUP BY gender, birth_year")
df.columns = ['gender', 'birth_year', 'count']
fig = px.bar(df, x='birth_year', y='count', color='gender')
fig.show()

The genders are quite equally distributed across birth years.

In [6]:
df = run_query('SELECT district_id, COUNT(*) count FROM client GROUP BY district_id')
df.columns = ['district_id', 'count']
fig = px.histogram(df, x='district_id', y='count')
fig.show()

Most of the clients are from district 0 to 9.

In [9]:
df = run_query("SELECT district_id, gender, COUNT(*) FROM client GROUP BY district_id, gender")
df.columns = ['district_id', 'gender', 'count']
fig = px.bar(df, x='district_id', y='count', color='gender')
fig.show()

The genders are equally distributed across districts.

### Disposition

In [14]:
df = run_query('SHOW COLUMNS FROM disp')
df

,0,1,2,3,4,5
0,disp_id,int(11),NO,PRI,None,
1,client_id,int(11),NO,MUL,None,
2,account_id,int(11),NO,MUL,None,
3,type,varchar(9),NO,,None,


In [15]:
df = run_query("SELECT type, COUNT(*) FROM disp GROUP BY type")
df.columns = ['type', 'count']
fig = px.bar(df, x='type', y='count')
fig.show()

Most of the dispositions are owner.